# File description
fourth and last step in the preprocessing. This file takes the cropped videos from `reducer3_crop.ipynb` remove all frames that don't contain humans and/or bikes. This is accomplished using 
[Ultralytic's implementation](https://github.com/ultralytics/yolov3)
of the object detection model YOLOv3 which is provided trough `pytorch`'s model zoo.
<br><br>
NOTE: YOLO's confidence score is set to the relatively low value of 25\% which hopefully ensures no frames a missed, but also means some false positives are included.

In [1]:
import dutils as U
U.jupyter_ipython.adjust_screen_width()

from tqdm.notebook import tqdm
from glob import glob
import warnings
import random
import torch
import cv2
import os
import numpy as np

# Constants

In [2]:
OLD_FOLDER_PATH = "C:/Users/JK/Desktop/reduced_crop" # Path to the original videos
NEW_FOLDER_PATH = "C:/Users/JK/Desktop/reduced_yolo" # Path where the processed videos are to be saved
TEMP_FOLDER_PATH = "C:/Users/JK/Desktop/temp_folders" # Path which temporarily contain frames while processing
DEVICE = U.pytorch.get_device()
BATCH_SIZE = 60
MODEL = torch.hub.load('ultralytics/yolov3', 'yolov3').to(DEVICE).eval()
MODEL.conf = 0.25

Using cache found in C:\Users\JK/.cache\torch\hub\ultralytics_yolov3_master
YOLOv3  2022-2-8 torch 1.8.1+cu111 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)

Fusing layers... 
Model Summary: 261 layers, 61922845 parameters, 0 gradients, 156.1 GFLOPs
Adding AutoShape... 


# AI Reduce Function
Function description:

    1.) Split the video at `video_path` into individual frames and store them in a new temporary folder
    2.) Divides these frames into batches and send them through YOLO
    3.) Keep only the frame that contains at least 1 human and/or 1 bike
    4.) Combine the selected frames into a new video which is saved at `save_video_path`
    5.) The temporary folder is deleted

In [6]:
def keep_frames(video_path:str, save_video_path:str, model, batch_size:int, temp_folder_path:str) -> None:
    # Checks
    assert os.path.exists(video_path), "Received bad path"
    assert os.path.exists(temp_folder_path) and os.path.isdir(temp_folder_path), "Received bad folder path"
    
    # Split video into frames (it's probably unwise to keep the frames in RAM, so will store them in temporary folder)
    temp_path = os.path.join(temp_folder_path, str(random.getrandbits(128)))
    os.mkdir(temp_path)
    U.videos.video_to_images(video_path, temp_path)

    # Create "batches" of paths pointing to the images located in the temporary folder created above
    paths = sorted(glob(os.path.join(temp_path, "*.png")), key=os.path.getmtime)
    batch, batches = [], []
    for i, p in enumerate(paths):
        batch.append(p)
        if i and (((i + 1) % batch_size == 0) or ((i + 1) == len(paths))):
            batches.append(batch)
            batch = []

    # Inference. Check if a human or bicycle is present in the image
    # NOTE: 0 and 1 encode person and cycle respectively in COCO
    findings = []
    for batch in batches:
        results = model(batch)
        
        for p in results.pred:
            if any(p[:, 5] < 2): # Keep all frames with at least one human and/or bicycle
                findings.append(True)
            else:
                findings.append(False)

    # Remove all frames without a human or a bicycle
    assert len(findings) == len(paths), "Cannot see how this should be legitimate"
    final_image_paths = [path for i, path in enumerate(paths) if findings[i]]
    
    # If the YOLO found 0 or 1 human --> Add empty/single frame and suffixes the saved file with "_EMPTY"
    if len(final_image_paths) < 2:
        if len(final_image_paths) == 1: print(save_video_path, " only contains 1 frame")
        grey_image_path = os.path.join(temp_folder_path, "dummy_image.png")
        info = U.videos.get_video_info(video_path)
        cv2.imwrite(grey_image_path, np.ones((info["height"], info["width"], 3)) * 150)
        final_image_paths += [grey_image_path for _ in range(5)] # Add 1 seconds of empty video
        save_video_path = new_video_path[:-4]+"_EMPTY"+".MP4"
    
    # Make the final video from the all frames containing humans/bicycles    
    U.videos.images_to_video(final_image_paths, save_video_path, fps=5)

    # Remove all temporary files
    [os.remove(path) for path in sorted(glob(os.path.join(temp_path, "*.png")), key=os.path.getmtime)] 
    try: 
        os.rmdir(temp_path)
    except PermissionError: 
        warnings.warn(f"PermissionError: Was unable to delete the temporary folder at: `{temp_path}`. Remove it manually.")

# Folders

In [7]:
old_folder_paths = [os.path.abspath(p) for p in glob(os.path.join(OLD_FOLDER_PATH, "*"))]
new_folder_paths = [os.path.join(NEW_FOLDER_PATH, os.path.basename(p)) for p in old_folder_paths]
for new_folder_path in new_folder_paths:
    if os.path.exists(new_folder_path): continue
    print(new_folder_path)
    assert not os.path.exists(new_folder_path), "Folder already exists"
    assert not os.path.isdir(new_folder_path), "Received non-folder path"
    os.mkdir(new_folder_path)

C:/Users/JK/Desktop/corrected/temp_18_human\Valby_18-12-2021


# Copy files

In [ ]:
for (old_folder_path, new_folder_path) in tqdm(list(zip(old_folder_paths, new_folder_paths))):
    for old_video_path in tqdm(glob(os.path.join(old_folder_path, "*"))):
        new_video_path = os.path.join(new_folder_path, os.path.basename(old_video_path))
        
        if os.path.exists(new_video_path) or os.path.exists(new_video_path[:-4]+"_EMPTY.MP4"): continue
        
        # Remove all frames without a human or a bicycle
        keep_frames(
            video_path = old_video_path,
            save_video_path = new_video_path,
            model = MODEL,
            batch_size = BATCH_SIZE,
            temp_folder_path = TEMP_FOLDER_PATH
        )